In [1]:
using Revise
using QuantumOptimalControl
using QuantumOptics
using LinearAlgebra
using Flux, DiffEqFlux
using CUDA
using CUDA.CUSPARSE
using Optim
using PlotlyJS
using ProgressMeter
using Random
using DifferentialEquations: DP5, Tsit5, BS3, Vern7
ProgressMeter.ijulia_behavior(:clear)

WebIO._IJuliaInit()

┌ Info: Precompiling QuantumOptimalControl [e91afccf-c93e-44fd-aa4c-3d3ef13645c0]
└ @ Base loading.jl:1423


false

In [2]:
bs = SpinBasis(1//2)
sx = sigmax(bs)
ni = 0.5*(identityoperator(bs) + sigmaz(bs));

In [3]:
V = 2π*24.0
δe = -2π*4.5

-28.274333882308138

In [4]:
n_atoms = 12
bsys = tensor([bs for i in 1:n_atoms]...)

H0 = V*sum([embed(bsys, [i, j], [ni, ni])/abs(i-j)^6  for i in 1:n_atoms for j in i+1:n_atoms])
H0 -= δe*sum([embed(bsys, [i], [ni]) for i in [1, n_atoms]])
if n_atoms>8
    H0 -= -2π*1.5*sum([embed(bsys, [i], [ni]) for i in [1, n_atoms]])
    H0 -= -2π*1.5*sum([embed(bsys, [i], [ni]) for i in [4, n_atoms-3]])
end;

H1 = 0.5*sum([embed(bsys, [i], [sx]) for i in 1:n_atoms])
H2 = -sum([embed(bsys, [i], [ni]) for i in 1:n_atoms]);

In [5]:
function GHZ_state(n_atoms)
    state = tensor([spindown(bs)⊗spinup(bs) for i in 1:Int(n_atoms/2)]...) +
            tensor([spinup(bs)⊗spindown(bs) for i in 1:Int(n_atoms/2)]...)
    state/sqrt(2.0)
end 

ground_state(n_atoms) = tensor([spindown(bs) for i in 1:n_atoms]...)
trans = StateTransform(ground_state(n_atoms)=>GHZ_state(n_atoms));

In [6]:
n_neurons = 8
sigmoid(x)= @. 2π*7 / (1 + exp(-x))
Random.seed!(10)
ann = FastChain(FastDense(1, n_neurons, tanh), 
                FastDense(n_neurons, n_neurons, tanh), 
                FastDense(n_neurons, 2))
θ = initial_params(ann)  
n_params = length(θ)

106

In [7]:
t0, t1 = 0.0, 0.5

tsf32 = Float32(t0):Float32(t1/49):Float32(t1)
Ωs = Vector{Float32}(2π*vcat(0:0.5:4, 5*ones(32), 4:-0.5:0))
Δs = Vector{Float32}(2π*(-5:10/49:5))
ts = Vector{Float64}(tsf32)

function loss(p)
    c = 0.0f0
    for (i,t) in enumerate(tsf32)
        x = ann([t], p)
        c += (abs(x[1]) - Ωs[i])^2
        c += (x[2] - Δs[i])^2
    end
    #println(c)
    c
end

res = DiffEqFlux.sciml_train(loss, initial_params(ann), ADAM(0.1f0), maxiters = 5000)
θ = Vector{Float64}(res.u);

In [8]:
coeffs(params, t) = let vals = ann([t], params)
                        [abs(vals[1]), vals[2]]
                    end    

cost = CostFunction((x, y) -> 1.0 - abs(sum(conj(x).*y)),
                     p->2e-3*(abs(ann([t0], p)[1])+ 5.0*abs(ann([t1], p)[1])))

CostFunction(var"#23#25"(), var"#24#26"())

In [9]:
H = Hamiltonian(H0, [H1, H2], coeffs);

In [10]:
prob = CuQOCProblem(QOCProblem(H, trans, (t0, t1), cost));

In [16]:
sol = solve(prob, Vector{Float64}(θ), ADAM(0.1); maxiter=200, abstol=1e-5, reltol=1e-5)

Progress: 100%|█████████████████████████████████████████| Time: 0:37:12
  distance:     0.01566833815006219
  constraints:  0.0038326262654798386


Solution{Float64}([-4.905475986299668, -3.2618115953157027, 3.6413560094651016, -4.3273473804535145, -9.812822771245065, -8.150170482722327, 8.853838784407298, -3.591286748354685, 0.9403170762968937, 1.4832495851028658  …  3.556977933882962, -6.402415433817247, 7.129873728447467, -7.192443977506046, 1.7158089689453633, 17.91182152591197, 19.364709688636616, 3.534464889928378, 1.9758769626795853, 6.1968074670422775], [0.8916733361705282, 0.8163625503099785, 0.8569844705917439, 0.7675663922933361, 0.8012910708456449, 0.8179537743189674, 0.7244935376324436, 0.6942092933902639, 0.6939731066378413, 0.6438212589418784  …  0.017141007857951807, 0.019621886146572565, 0.01941471005219708, 0.01712336110721968, 0.015512712242200855, 0.017261555678665608, 0.01870360223769174, 0.0206763887875121, 0.016361573292679066, 0.01566833815006219])

In [17]:
plot(sol.trace)

data: [
  "scatter with fields type, x, and y"
]

layout: "layout with fields margin and template"

In [18]:
Ω(t) = abs(ann([t], sol.params)[1])/2π
Δ(t) = ann([t], sol.params)[2]/2π
ts = collect(t0:0.001:t1)

f = plot(
    [
        scatter(x=ts, y=Ω.(ts), name="Ω/2π"),
        scatter(x=ts, y=Δ.(ts), name="Δ/2π"),
    ],
    Layout(
        xaxis_title_text="Time (µs)",
        yaxis_title_text="Frequency (MHz)",
        legend=attr(x=0, y=1,),
        font=attr(
            size=16
        )
    )
)
savefig(f, "GHZ_12_atoms_wfs.eps")

"GHZ_12_atoms_wfs.eps"

In [19]:
tout, psit = schroedinger_dynamic(ts, ground_state(n_atoms), H, sol.params);

In [20]:
f = plot(
    [
        scatter(x=ts, y=real(expect(dm(GHZ_state(n_atoms)), psit))),
    ],
    Layout(
        xaxis_title_text="Time (µs)",
        yaxis_title_text="Overlap (|⟨ψ|GHZ⟩|²)",
        font=attr(
            size=16
        )
    )
)
savefig(f,"GHZ_12_atoms_overlap.eps")

"GHZ_12_atoms_overlap.eps"